# Tutorial #7: Develop a feature set using Domain Specific Language (preview)

Domain Specific Language (DSL) for the managed feature store provides a simple and user-friendly way of defining most commonly used feature aggregations. The `featurestore` SDK allows users to perform most commonly used aggregations by using a DSL *expression*. These aggregations ensure consistent results when compared with user-defined functions (UDFs) without the overhead of writing UDFs.

In this tutorial you will:
- Create a new minimal feature store workspace.
- Develop and test feature set locally by using Domain Specific Language (DSL).
- Develop a feature set by using User Defined Functions (UDFs) that performs the same transformation as the feature set created using DSL.
- Compare results from the feature sets created using DSL and UDFs.
- Register a feature store entity with the feature store.
- Register the feature set created using DSL with the feature store.
- Generate sample training data using the created features.

#### Important

This feature is currently in private preview. This preview version is provided without a service-level agreement, and it's not recommended for production workloads. Certain features might not be supported or might have constrained capabilities. For more information, see [Supplemental Terms of Use for Microsoft Azure Previews](https://azure.microsoft.com/support/legal/preview-supplemental-terms/).

## Prerequisites

> [!NOTE]
> This tutorial uses Azure Machine Learning notebook with **Serverless Spark Compute**.

Before following the steps in this article, make sure you have the following prerequisites:

1. An Azure Machine Learning workspace. If you don't have one, use the steps in the [Quickstart: Create workspace resources](https://learn.microsoft.com/azure/machine-learning/quickstart-create-resources?view=azureml-api-2) article to create one.
1. To perform the steps in this article, your user account must be assigned the **Owner** or **Contributor** role to the resource group where the feature store will be created.

#### Prepare the notebook environment for development

1. Download a zip file from the [`featurestore-dsl` branch of examples repository (azureml-examples)](https://github.com/Azure/azureml-examples/tree/featurestore-dsl): click on the `code` dropdown and select `Download ZIP`. Then unzip the contents into a folder in your local machine.

2. Upload the **featurestore_sample** directory to project workspace: 
   1. Open Azure Machine Learning studio UI of your Azure Machine Learning workspace 
   2. Select on **Notebooks** in left navigation.
   3. Right click on your user name in the directory listing.
   4. Select **Upload folder**.
   5. Select the **featurestore_sample** folder from the cloned directory path: `azureml-examples/sdk/python/featurestore_sample`

3. You can either create a new notebook and copy and paste code blocks in this notebook step by step and execute OR open the existing notebook `azureml-examples/sdk/python/featurestore_sample/notebooks/sdk_only/7. Feature store Domain Specific Language.ipynb` and execute step by step. Keep this document open and refer to it for detailed explanation of the steps.

4. In the **Compute** dropdown in the top navigation, select **Serverless Spark Compute**. It may take 1-2 minutes for serverless Spark compute to attach to the notebook. Wait for a status bar in the top to display **Configure session**.

5. To configure Spark session:
   1. Select **Configure session**.
   2. Select **Settings**.
   3. Select **Apache Spark version** as `Spark version 3.3`.
   4. Optionally, increase the **Session timeout** (idle time) if you want to avoid frquently restarting the serverless Spark session.
   5. Under **Configuration settings**, define *Property* `spark.jars` and *Value* `wasbs://featurestore@azuremlexamples.blob.core.windows.net/jars/fs-scala-impl.jar`.
      ![DSL_SPARK_JARS_PROPERTY](./images/dsl-spark-jar-property.png) 
   6. Select **Python packages**.
   7. Select **Upload conda file**.
   8. Select the file `azureml-examples/sdk/python/featurestore_sample/project/env/dsl.yml` from your local machine.
   9.  Select **Apply**.

__Important:__ Except for this step, you need to run all the other steps every time you start a new Spark session/session time out.


## Setup root directory for the samples and enable preview features
This code cell sets up the root directory for the samples and set environment variable to enable preview features. It may needs about 10 minutes to execute this cell as it also installs all Conda dependencies and starts the Spark session.

In [ ]:
import os

# Please update your alias belpw (or any custom directory you uploaded the samples to).
# You can find the name from the directory structure in the left nav
root_dir = "./Users/<USER_NAME>/featurestore_sample"

if os.path.isdir(root_dir):
    print("The folder exists.")
else:
    print("The folder does not exist. Please create or fix the path")

## Create a minimal feature store

Create a feature store in **East US 2 EUAP** region from the Azure Machine Learning Studio UI or using SDK code. 

### Option 1. Create feature store from the Azure Machine Learning Studio UI

1. Navigate to the feature store UI [landing page](https://ml.azure.com/featureStores).
2. Select **+ Create**.
3. On the **Basics** tab:
   1. Choose a name for your feature store.
   2. Select the **Subscription**. 
   3. Select the **Resource group**.
   4. Select the **Region** __East US 2 EUAP__.
   5. Select **Apache Spark version** 3.3. then select **Next**.
4. On the **Materialization** tab:
   1. Toggle **Enable materialization**.
   2. Select **Subscription** and **User identity** to **Assign user managed identity**.
   3. Select **From Azure subscription** under **Offline store**.
   4. Select **Store name** and **Azure Data Lake Gen2 file system name**, then select **Next**.
5. On the **Review** tab, verify the displayed information and then select **Create**.

### Option 2. Create feature store from the Python SDK
Provide `featurestore_name`, `featurestore_resource_group_name`, and `featurestore_subscription_id` and execute the following cell to create a minimal feature store.

In [ ]:
import os

featurestore_name = "<FEATURE_STORE_NAME>"
featurestore_resource_group_name = "<RESOURCE_GROUP>"
featurestore_subscription_id = "<SUBSCRIPTION_ID>"

##### Create Feature Store #####
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    FeatureStore,
    FeatureStoreEntity,
    FeatureSet,
)
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

ml_client = MLClient(
    AzureMLOnBehalfOfCredential(),
    subscription_id=featurestore_subscription_id,
    resource_group_name=featurestore_resource_group_name,
)
featurestore_location = "eastus2euap"

fs = FeatureStore(name=featurestore_name, location=featurestore_location)
# wait for featurestore creation
fs_poller = ml_client.feature_stores.begin_create(fs)
print(fs_poller.result())

### Grant "Storage Blob Data Reader" role to your user identity on the offline store
If feature data is materialized, then you need this role to read feature data from offline materialization store.
- Open the [Azure ML global landing page](https://ml.azure.com/home).
- Select **Feature stores** in the left navigation.
- You will see the list of feature stores that you have access to. Select the feature store that you have created above.
- Select storage account link under **Account name** on **Offline materialization store** card to navigate to ADLS Gen2 storage account for offline store. 
![OFFLINE_STORE_INFO](./images/offline-store-information.png)
- Follow [this documentation](https://learn.microsoft.com/azure/role-based-access-control/role-assignments-portal) to assign **Storage Blob Data Reader** role to your user identity on the ADLS Gen2 storage account for offline store. Allow some time for permissions to propagate.

## Create a feature set specification using DSL expressions
Execute the following code cell to create a feature set specification using DSL expressions. Currently, the following aggregation expressions are supported:
- Average - `avg`
- Sum - `sum`
- Minimum - `min`
- Maximum - `max`
- Count - `count`

In [ ]:
from azureml.featurestore import create_feature_set_spec
from azureml.featurestore.contracts.feature import Feature
from azureml.featurestore.transformation import (
    TransformationExpressionCollection,
    WindowAggregation,
)
from azureml.featurestore.contracts import (
    DateTimeOffset,
    FeatureSource,
    TransformationCode,
    Column,
    ColumnType,
    SourceType,
    TimestampColumn,
)

dsl_feature_set_spec = create_feature_set_spec(
    source=FeatureSource(
        type=SourceType.parquet,
        path="wasbs://data@azuremlexampledata.blob.core.windows.net/feature-store-prp/datasources/transactions-source/*.parquet",
        timestamp_column=TimestampColumn(name="timestamp"),
    ),
    index_columns=[Column(name="accountID", type=ColumnType.string)],
    features=[
        Feature(name="f_transaction_3d_count", type=ColumnType.Integer),
        Feature(name="f_transaction_amount_3d_sum", type=ColumnType.DOUBLE),
        Feature(name="f_transaction_amount_3d_avg", type=ColumnType.DOUBLE),
        Feature(name="f_transaction_7d_count", type=ColumnType.Integer),
        Feature(name="f_transaction_amount_7d_sum", type=ColumnType.DOUBLE),
        Feature(name="f_transaction_amount_7d_avg", type=ColumnType.DOUBLE),
    ],
    feature_transformation=TransformationExpressionCollection(
        transformation_expressions=[
            WindowAggregation(
                feature_name="f_transaction_3d_count",
                aggregation="count",
                window=DateTimeOffset(days=3),
            ),
            WindowAggregation(
                feature_name="f_transaction_amount_3d_sum",
                source_column="transactionAmount",
                aggregation="sum",
                window=DateTimeOffset(days=3),
            ),
            WindowAggregation(
                feature_name="f_transaction_amount_3d_avg",
                source_column="transactionAmount",
                aggregation="avg",
                window=DateTimeOffset(days=3),
            ),
            WindowAggregation(
                feature_name="f_transaction_7d_count",
                aggregation="count",
                window=DateTimeOffset(days=7),
            ),
            WindowAggregation(
                feature_name="f_transaction_amount_7d_sum",
                source_column="transactionAmount",
                aggregation="sum",
                window=DateTimeOffset(days=7),
            ),
            WindowAggregation(
                feature_name="f_transaction_amount_7d_avg",
                source_column="transactionAmount",
                aggregation="avg",
                window=DateTimeOffset(days=7),
            ),
        ]
    ),
)

dsl_feature_set_spec

Define a feature window start date and end date.

In [ ]:
from datetime import datetime

st = datetime(2020, 1, 1)
et = datetime(2023, 6, 1)

Use `to_spark_dataframe()` to get a dataframe from the above feature set specification defined using DSL expressions.

In [ ]:
dsl_df = dsl_feature_set_spec.to_spark_dataframe(
    feature_window_start_date_time=st, feature_window_end_date_time=et
)

In [ ]:
display(dsl_df)

## Create a feature set specification with Custom Source using DSL expressions
Execute the following code cell to create a feature set specification using DSL expressions. Currently, the following aggregation expressions are supported:
- Average - `avg`
- Sum - `sum`
- Minimum - `min`
- Maximum - `max`
- Count - `count`

In [ ]:
from azureml.featurestore import create_feature_set_spec
from azureml.featurestore.contracts.feature import Feature
from azureml.featurestore.feature_source import CustomFeatureSource
from azureml.featurestore.transformation import (
    TransformationExpressionCollection,
    WindowAggregation,
)
from azureml.featurestore.contracts import (
    DateTimeOffset,
    FeatureSource,
    SourceProcessCode,
    TransformationCode,
    Column,
    ColumnType,
    SourceType,
    TimestampColumn,
)

transactions_source_process_code_path = (
    root_dir
    + "/featurestore/featuresets/transactions_custom_source/source_process_code"
)

dsl_custom_feature_set_spec = create_feature_set_spec(
    source=CustomFeatureSource(
        kwargs={
            "source_path": "wasbs://data@azuremlexampledata.blob.core.windows.net/feature-store-prp/datasources/transactions-source-json/*.json",
            "timestamp_column_name": "timestamp",
        },
        timestamp_column=TimestampColumn(name="timestamp"),
        source_delay=DateTimeOffset(days=0, hours=0, minutes=20),
        source_process_code=SourceProcessCode(
            path=transactions_source_process_code_path,
            process_class="source_process.CustomSourceTransformer",
        ),
    ),
    index_columns=[Column(name="accountID", type=ColumnType.string)],
    features=[
        Feature(name="f_transaction_3d_count", type=ColumnType.Integer),
        Feature(name="f_transaction_amount_3d_sum", type=ColumnType.DOUBLE),
        Feature(name="f_transaction_amount_3d_avg", type=ColumnType.DOUBLE),
        Feature(name="f_transaction_7d_count", type=ColumnType.Integer),
        Feature(name="f_transaction_amount_7d_sum", type=ColumnType.DOUBLE),
        Feature(name="f_transaction_amount_7d_avg", type=ColumnType.DOUBLE),
    ],
    feature_transformation=TransformationExpressionCollection(
        transformation_expressions=[
            WindowAggregation(
                feature_name="f_transaction_3d_count",
                aggregation="count",
                window=DateTimeOffset(days=3),
            ),
            WindowAggregation(
                feature_name="f_transaction_amount_3d_sum",
                source_column="transactionAmount",
                aggregation="sum",
                window=DateTimeOffset(days=3),
            ),
            WindowAggregation(
                feature_name="f_transaction_amount_3d_avg",
                source_column="transactionAmount",
                aggregation="avg",
                window=DateTimeOffset(days=3),
            ),
            WindowAggregation(
                feature_name="f_transaction_7d_count",
                aggregation="count",
                window=DateTimeOffset(days=7),
            ),
            WindowAggregation(
                feature_name="f_transaction_amount_7d_sum",
                source_column="transactionAmount",
                aggregation="sum",
                window=DateTimeOffset(days=7),
            ),
            WindowAggregation(
                feature_name="f_transaction_amount_7d_avg",
                source_column="transactionAmount",
                aggregation="avg",
                window=DateTimeOffset(days=7),
            ),
        ]
    ),
)

dsl_custom_feature_set_spec

In [ ]:
custom_source_dsl_df = dsl_custom_feature_set_spec.to_spark_dataframe(
    feature_window_start_date_time=st, feature_window_end_date_time=et
)

In [ ]:
display(custom_source_dsl_df)

## Create a feature set specification using UDF
Now, create a feature set specification that performs the same transformations using UDF.

In [ ]:
from azureml.featurestore import create_feature_set_spec
from azureml.featurestore.contracts import (
    DateTimeOffset,
    FeatureSource,
    TransformationCode,
    Column,
    ColumnType,
    SourceType,
    TimestampColumn,
)

transactions_featureset_code_path = (
    root_dir + "/featurestore/featuresets/transactions/transformation_code"
)

udf_feature_set_spec = create_feature_set_spec(
    source=FeatureSource(
        type=SourceType.parquet,
        path="wasbs://data@azuremlexampledata.blob.core.windows.net/feature-store-prp/datasources/transactions-source/*.parquet",
        timestamp_column=TimestampColumn(name="timestamp"),
        source_delay=DateTimeOffset(days=0, hours=0, minutes=20),
    ),
    transformation_code=TransformationCode(
        path=transactions_featureset_code_path,
        transformer_class="transaction_transform.TransactionFeatureTransformer",
    ),
    index_columns=[Column(name="accountID", type=ColumnType.string)],
    infer_schema=True,
)

udf_feature_set_spec

The transformation code is provided here to show that the UDF defines same transformations as the DSL expressions.

```python
class TransactionFeatureTransformer(Transformer):
    def _transform(self, df: DataFrame) -> DataFrame:
        days = lambda i: i * 86400
        w_3d = (
            Window.partitionBy("accountID")
            .orderBy(F.col("timestamp").cast("long"))
            .rangeBetween(-days(3), 0)
        )
        w_7d = (
            Window.partitionBy("accountID")
            .orderBy(F.col("timestamp").cast("long"))
            .rangeBetween(-days(7), 0)
        )
        res = (
            df.withColumn("transaction_7d_count", F.count("transactionID").over(w_7d))
            .withColumn(
                "transaction_amount_7d_sum", F.sum("transactionAmount").over(w_7d)
            )
            .withColumn(
                "transaction_amount_7d_avg", F.avg("transactionAmount").over(w_7d)
            )
            .withColumn("transaction_3d_count", F.count("transactionID").over(w_3d))
            .withColumn(
                "transaction_amount_3d_sum", F.sum("transactionAmount").over(w_3d)
            )
            .withColumn(
                "transaction_amount_3d_avg", F.avg("transactionAmount").over(w_3d)
            )
            .select(
                "accountID",
                "timestamp",
                "transaction_3d_count",
                "transaction_amount_3d_sum",
                "transaction_amount_3d_avg",
                "transaction_7d_count",
                "transaction_amount_7d_sum",
                "transaction_amount_7d_avg",
            )
        )
        return res

```

Use `to_spark_dataframe()` to get a dataframe from the above feature set specification defined using UDF.

In [ ]:
udf_df = udf_feature_set_spec.to_spark_dataframe(
    feature_window_start_date_time=st, feature_window_end_date_time=et
)

Compare the results and verify consistency between the results from DSL expressions and transformations performed using UDF. To verify, select one of the `accountID` values to compare the values in the two dataframes.

In [ ]:
display(dsl_df.where(dsl_df.accountID == "A1899946977632390").sort("timestamp"))

In [ ]:
display(udf_df.where(udf_df.accountID == "A1899946977632390").sort("timestamp"))

### Export as feature set specification
In order to register the feature set specification with the feature store, it needs to be saved in a specific format. 
Action: Please inspect the generated `transactions-dsl` feature set specification: Open this file from the file tree to see the specification: `featurestore/featuresets/transactions-dsl/spec/FeaturesetSpec.yaml`

The feature set specification contains these important elements:

1. `source`: Reference to a storage. In this case a parquet file in a blob storage.
2. `features`: List of features and their datatypes. If you provide transformation code, the code has to return a dataframe that maps to the features and data types.
3. `index_columns`: The join keys required to access values from the feature set

Learn more about it in the [top level feature store entities document](https://learn.microsoft.com/azure/machine-learning/concept-top-level-entities-in-managed-feature-store) and the [feature set specification YAML reference](https://learn.microsoft.com/azure/machine-learning/reference-yaml-featureset-spec).

The additional benefit of persisting the feature set specification is that it can be source controlled.

In [ ]:
dsl_spec_folder = root_dir + "/featurestore/featuresets/transactions-dsl/spec"

dsl_feature_set_spec.dump(dsl_spec_folder, overwrite=True)

In [ ]:
custom_source_dsl_spec_folder = (
    root_dir + "/featurestore/featuresets/transactions-dsl/custom_source_spec"
)

dsl_custom_feature_set_spec.dump(custom_source_dsl_spec_folder, overwrite=True)

In [ ]:
udf_spec_folder = root_dir + "/featurestore/featuresets/transactions-udf/spec"

udf_feature_set_spec.dump(udf_spec_folder, overwrite=True)

## Initialize feature store workspace CRUD client and feature store core SDK client 

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    FeatureStore,
    FeatureStoreEntity,
    FeatureSet,
)
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential
from azureml.featurestore import FeatureStoreClient

fs_client = MLClient(
    AzureMLOnBehalfOfCredential(),
    featurestore_subscription_id,
    featurestore_resource_group_name,
    featurestore_name,
)

featurestore = FeatureStoreClient(
    credential=AzureMLOnBehalfOfCredential(),
    subscription_id=featurestore_subscription_id,
    resource_group_name=featurestore_resource_group_name,
    name=featurestore_name,
)

## Register `account` entity with the feature store
Create account entity that has join key `accountID` of `string` type. 

In [ ]:
from azure.ai.ml.entities import DataColumn

account_entity_config = FeatureStoreEntity(
    name="account",
    version="1",
    index_columns=[DataColumn(name="accountID", type="string")],
)

poller = fs_client.feature_store_entities.begin_create_or_update(account_entity_config)
print(poller.result())

## Register the feature set with the feature store using the exported feature set specification
Register the `transactions-dsl` feature set with the feature store with offline materialization enabled using the exported feature set specification.

In [ ]:
from azure.ai.ml.entities import (
    FeatureSet,
    FeatureSetSpecification,
    MaterializationSettings,
    MaterializationComputeResource,
)

materialization_settings = MaterializationSettings(
    offline_enabled=True,
    resource=MaterializationComputeResource(instance_type="standard_e8s_v3"),
    spark_configuration={
        "spark.driver.cores": 4,
        "spark.driver.memory": "36g",
        "spark.executor.cores": 4,
        "spark.executor.memory": "36g",
        "spark.executor.instances": 2,
    },
    schedule=None,
)

fset_config = FeatureSet(
    name="transactions-dsl",
    version="1",
    entities=["azureml:account:1"],
    stage="Development",
    specification=FeatureSetSpecification(path=dsl_spec_folder),
    materialization_settings=materialization_settings,
    tags={"data_type": "nonPII"},
)

poller = fs_client.feature_sets.begin_create_or_update(fset_config)
print(poller.result())

Materialize the feature set to persist the transformed feature data to the offline store.

In [ ]:
spark_config = """
dependencies:
  - python=3.8
  - pip=21.1.1
  - pip:
    - mlflow
    - azureml-mlflow
    - protobuf==3.19.6
    - azureml-featurestore[online] @ https://featurestoreprp.blob.core.windows.net/test-sdk/runhli_test/dsl/azureml_featurestore-0.0.1-py3-none-any.whl
    - aiohttp==3.8.4
    - azure-mgmt-msi
name: fs_env
"""


poller = fs_client.feature_sets.begin_backfill(
    name="transactions-dsl",
    version="1",
    feature_window_start_time=st,
    feature_window_end_time=et,
    spark_configuration={
        "spark.synapse.library.python.env": spark_config,
        "spark.jars": "https://featurestoreprp.blob.core.windows.net/test-sdk/runhli_test/dsl/azureml-fs-scala-impl-1.0.0.jar",
    },
    data_status=["None", "Incomplete"],
)
print(poller.result().job_ids)

Track the progress of materialization job.

In [ ]:
# get the job URL, and stream the job logs (the back fill job could take 10+ minutes to complete)
fs_client.jobs.stream(poller.result().job_ids[0])

Print sample data from the feature set. Notice from the output information that the data was retrieved from the materilization store. `get_offline_features()` method that is used to retrieve training/inference data will also use the materialization store by default.

In [ ]:
# look up the featureset by providing name and version
transactions_featureset = featurestore.feature_sets.get("transactions-dsl", "1")
display(transactions_featureset.to_spark_dataframe().head(5))

## Generate a training dataframe using the registered feature set

### Load observation data

Start by exploring the observation data. Observation data is typically the core data used in training and inference steps. This is then joined with the feature data to create complete training data. Observation data is the data captured during the time of the event: in this case it has core transaction data including transaction ID, account ID, transaction amount. Since this data is used for training, it also has the target variable appended (`is_fraud`).

In [ ]:
observation_data_path = "wasbs://data@azuremlexampledata.blob.core.windows.net/feature-store-prp/observation_data/train/*.parquet"
observation_data_df = spark.read.parquet(observation_data_path)
obs_data_timestamp_column = "timestamp"

display(observation_data_df)
# Note: the timestamp column is displayed in a different format. Optionally, you can can call training_df.show() to see correctly formatted value

Select features that would be part of the training data and use the feature store SDK to generate the training data.

In [ ]:
featureset = featurestore.feature_sets.get("transactions-dsl", "1")

# you can select features in pythonic way
features = [
    featureset.get_feature("f_transaction_amount_7d_sum"),
    featureset.get_feature("f_transaction_amount_7d_avg"),
]

# you can also specify features in string form: featurestore:featureset:version:feature
more_features = [
    "transactions-dsl:1:f_transaction_amount_3d_sum",
    "transactions-dsl:1:f_transaction_3d_count",
]

more_features = featurestore.resolve_feature_uri(more_features)
features.extend(more_features)

`get_offline_features()` appends the features to the observation data using a point-in-time join. Display the training dataframe obtained from the point-in-time join.

In [ ]:
from azureml.featurestore import get_offline_features

training_df = get_offline_features(
    features=features,
    observation_data=observation_data_df,
    timestamp_column=obs_data_timestamp_column,
)

display(training_df.sort("transactionID", "accountID", "timestamp"))

### Generate a training dataframe using registered feature sets with different types of transformations

register a udf feature set

In [ ]:
fset_config = FeatureSet(
    name="transactions-udf",
    version="1",
    entities=["azureml:account:1"],
    stage="Development",
    specification=FeatureSetSpecification(path=udf_spec_folder),
    materialization_settings=materialization_settings,
    tags={"data_type": "nonPII"},
)

poller = fs_client.feature_sets.begin_create_or_update(fset_config)
print(poller.result())

In [ ]:
featureset_dsl = featurestore.feature_sets.get("transactions-dsl", "1")
featureset_udf = featurestore.feature_sets.get("transactions-udf", "1")

# you can select features in pythonic way
features = [
    featureset_dsl.get_feature("f_transaction_amount_7d_sum"),
    featureset_udf.get_feature("transaction_amount_7d_avg"),
]

# you can also specify features in string form: featurestore:featureset:version:feature
more_features = [
    "transactions-udf:1:transaction_amount_3d_sum",
    "transactions-dsl:1:f_transaction_3d_count",
]

more_features = featurestore.resolve_feature_uri(more_features)
features.extend(more_features)

get_offline_features() appends the features to the observation data using a point-in-time join. Display the training dataframe obtained from the point-in-time join.

In [ ]:
from azureml.featurestore import get_offline_features

training_df = get_offline_features(
    features=features,
    observation_data=observation_data_df,
    timestamp_column=obs_data_timestamp_column,
)

display(training_df.sort("transactionID", "accountID", "timestamp"))

## Cleanup

Tutorial `5. Develop a feature set with custom source` has instructions for deleting the resources.

## Next steps
* Experiment and train models using features.